In [14]:
import advent
from typing import NamedTuple, Iterator
from advent.maze import solve_maze
from advent.utils import np_where_as_tuple, tadd2, tsub2
import numpy as np
advent.scrape(2024, 16, 'csv')
data = advent.get_char_grid(16)

reindeer = np_where_as_tuple(data, 'S')[0]
end = np_where_as_tuple(data, 'E')[0]

In [ ]:
class Node(NamedTuple):
    reindeer: tuple[int, int]
    offset: tuple[int, int] = (0, 1)

start = Node(reindeer)
is_end = lambda node: data[node.reindeer] == 'E'

def adjacent(node: Node, reverse=False) -> Iterator[tuple[Node, int]]:
    # Two possibilities: moving or turning
    move_position = tadd2(node.reindeer, node.offset)
    if reverse: move_position = tsub2(node.reindeer, node.offset) # just for part 2
    if data[move_position] != '#': yield (Node(move_position, node.offset), 1)
    # Try two different turns, they are the same in reverse
    new_offsets = [(-node.offset[1], node.offset[0]), (node.offset[1], -node.offset[0])]
    for offset in new_offsets:
        yield Node(node.reindeer, offset), 1000


length, _ = solve_maze(start, is_end, adjacent)


  0%|          | 0/1 [00:00<?, ?it/s]

Final path length: 98484


In [ ]:
# I think I just have to kind of brute force it ... (kinda sorta)
# Idea: calculate quickest route to every node, then quickest route from node to end
# The first one we already kinda did in part 1. The second one we can do with solve_maze_no_tqdm
# Note: this didn't work, was way too slow calling solve_maze tens of thousands of times
# instead, let's do the second one with dijkstra as well
# The rub: there are two possible end positions, so we have to check both

end1 = Node(end, (-1, 0))  # these offsets are hardcoded based on manual inspection
end2 = Node(end, (0, 1))
adjacent_reverse = lambda node: adjacent(node, True)

_, closed_parents_start = solve_maze(start, lambda n: False, adjacent)
_, closed_parents_end1 = solve_maze(end1, lambda n: False, adjacent_reverse)
_, closed_parents_end2 = solve_maze(end2, lambda n: False, adjacent_reverse)

In [17]:

from advent.maze import solve_maze_no_tqdm

def calculate_cost(node: Node, closed_parents: dict[Node, Node]) -> int:
    result, parent = 0, None
    while closed_parents[node] is not None:
        parent = closed_parents[node]
        if parent.reindeer == node.reindeer: result += 1000
        elif parent.offset == node.offset: result += 1
        else: raise ValueError()
        node = parent
    return result

costs_start = dict([(n, calculate_cost(n,closed_parents_start)) for n in closed_parents_start])
costs_end1 = dict([(n, calculate_cost(n,closed_parents_end1)) for n in closed_parents_end1])
costs_end2 = dict([(n, calculate_cost(n,closed_parents_end2)) for n in closed_parents_end2])

In [ ]:
result = set()
for n in costs_start:
    remaining_cost = min(costs_end1[n], costs_end2[n])  # Check both possible end positions
    total_cost = costs_start[n] + remaining_cost
    if total_cost == length:
        result.add(n.reindeer)  # we only care about unique coords not rotation
assert end1.reindeer in result and start.reindeer in result  # Sanity checks
print(len(result))

531
